In [ ]:
import re
import pandas as pd

In [ ]:
f = open('group.txt','r',encoding= 'utf-8')
# f = open('nongroup.txt','r',encoding= 'utf-8')

In [ ]:
data = f.read()

In [ ]:
print(data)

In [ ]:
pattern = r'\[(\d{2}/\d{2}/\d{2}, \d{2}:\d{2}:\d{2}\s*[AP]M)\] (.*?)$'

In [ ]:
messages = []

for match in re.findall(pattern, data, re.MULTILINE):
    messages.append(match[1].strip())
messages

In [ ]:
dates = []

for match in re.findall(pattern, data, re.MULTILINE):
    dates.append(match[0].strip())
dates

In [ ]:
df = pd.DataFrame({'message_date':dates,'user_message':messages})
df['message_date'] = pd.to_datetime(df['message_date'], format='%y/%m/%d, %I:%M:%S %p')

df.rename(columns={'message_date': 'date'}, inplace=True)

df.head()

In [ ]:
df.shape

In [ ]:
users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s',message)
    if entry[1:]: # user name
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()

In [ ]:
df['year'] = df['date'].dt.year

In [ ]:
df['month'] = df['date'].dt.month_name()

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df['hour'] = df['date'].dt.hour

In [ ]:
df['minute'] = df['date'].dt.minute

In [ ]:
df.head()

In [ ]:
df[df['user'] == '~ Sowmesh Sharma'].shape

In [ ]:
words = []
for message in df['message']:
    words.extend(message.split())

In [ ]:
len(words)

In [ ]:
df[df['message'] == '\u200Eimage omitted']

In [ ]:
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls("Let's www.gamil.com have URL stackoverflow.com as an example google.com, http://facebook.com, ftp://url.in")
urls

In [ ]:
links = []

for message in df['message']:
    links.extend(extractor.find_urls(message))

In [ ]:
len(links)

In [ ]:
df

In [ ]:
x = df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
name = x.index
count = x.values

In [ ]:
plt.bar(name,count)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
round((df['user'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'index':'name', 'user':'percent'})

In [ ]:
words = []

for message in df['message']:
    words.extend(message.split())

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

In [ ]:
# reomve grp notification
temp = df[df['user'] == 'group_notification']
temp = temp[temp['message'] != '\u200Eimage omitted']